Load Context and Questions

In [2]:
#ChatGPT
with open('/content/drive/MyDrive/Colab Notebooks/CapStone/context.txt', "r") as file:
    file_content = [line.strip() for line in file]

import pandas as pd

questionSet = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/questions.csv')
questiones = questionSet.values.tolist()


Load transformers

In [3]:
from transformers import pipeline
db_question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline


#GPT2
#https://huggingface.co/anas-awadalla/gpt-2-small-squad
model_name = "anas-awadalla/gpt-2-large-squad"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#gpt2_question_answer = pipeline('question-answering', model=model_name, tokenizer=model_name)


#T5 textgen
#https://huggingface.co/lmqg/t5-small-squad-qag
t5_question_answer = pipeline("text2text-generation", "lmqg/t5-small-squad-qag")

#Bert Squad
#https://huggingface.co/deepset/bert-base-cased-squad2
model_name = "deepset/bert-base-cased-squad2"

bert_squad_question_answer = pipeline('question-answering', model=model_name, tokenizer=model_name)


#Distil Bert
#https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad
db_question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


#Bert Base
#https://huggingface.co/google-bert/bert-base-uncased
model_name = "bert-base-cased"

bert_base_question_answer = pipeline('question-answering', model=model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at anas-awadalla/gpt-2-large-squad and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weigh

Generate Responses

In [4]:
gpt2_responses = []
t5_responses = []
bert_squad_responses = []
bert_base_responses = []
db_responses = []

for question in questiones:
  question_idx = question[0]
  context = file_content[question_idx]
  question_ask = question[1]
  answer = question[2]

  db_responses.append([question_ask, answer, db_question_answerer(question=question_ask, context=context)['answer']])
  #gpt2_responses.append([question_ask, answer, gpt2_question_answer(question=question_ask, context = context)['answer']])
  #t5_responses.append([question_ask, answer, t5_question_answer(f"{question_ask}, context = {context}")[0]['generated_text']])
  bert_squad_responses.append([question_ask, answer, bert_squad_question_answer({'question': question_ask, 'context': context})['answer']])
  bert_base_responses.append([question_ask, answer, bert_base_question_answer({'question': question_ask, 'context': context})['answer']])

pd.DataFrame(db_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/db_responses.csv')
#pd.DataFrame(gpt2_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/gpt2_responses.csv')
#pd.DataFrame(t5_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/t5_responses.csv')
pd.DataFrame(bert_squad_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_squad_responses.csv')
pd.DataFrame(bert_base_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_base_responses.csv')



/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Cosine Similarities

In [31]:
#(pratikchakra18)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample documents

# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer()

# Compute Cosine Similarity

db_responses = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/db_responses.csv').values.tolist()
bert_squad_responses = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_squad_responses.csv').values.tolist()
bert_base_responses = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_base_responses.csv').values.tolist()

for i in range(len(db_responses)):
  answers = tfidf_vectorizer.fit_transform((str(db_responses[i][1]), str(db_responses[i][2])))
  if str(db_responses[i][2]) == str(db_responses[i][3]):
    db_responses[i][5] = 1
  else:
    db_responses[i][5] = cosine_similarity(answers[0:1], answers)[0][1]

for i in range(len(bert_squad_responses)):
  answers = tfidf_vectorizer.fit_transform([str(bert_squad_responses[i][1]), str(bert_squad_responses[i][2])])
  if str(bert_squad_responses[i][1]) == str(bert_squad_responses[i][2]):
    bert_squad_responses[i][3] = 1
  else:
    bert_squad_responses[i][3] = cosine_similarity(answers[0:1], answers)[0][1]

for i in range(len(bert_base_responses)):
  answers = tfidf_vectorizer.fit_transform([str(bert_base_responses[i][1]), str(bert_base_responses[i][2])])
  if str(bert_base_responses[i][1]) == str(bert_base_responses[i][2]):
    bert_base_responses[i][3] = 1
  else:
    bert_base_responses[i][3] = cosine_similarity(answers[0:1], answers)[0][1]

In [19]:
pd.DataFrame(db_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/db_responses.csv')
pd.DataFrame(bert_squad_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_squad_responses.csv')
pd.DataFrame(bert_base_responses).to_csv('/content/drive/MyDrive/Colab Notebooks/CapStone/bert_base_responses.csv')

Individual models running on sample question

In [ ]:
from transformers import pipeline
db_question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

context = file_content[2]
question = "What is southern California often abbreviated as?"
result = db_question_answerer(question=question, context=context)
print(result['answer'])

SoCal


In [ ]:
#https://huggingface.co/anas-awadalla/gpt-2-small-squad
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "anas-awadalla/gpt-2-large-squad"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

qa = pipeline('question-answering', model=model_name, tokenizer=model_name)

question = "What is southern California often abbreviated as?"
qa(context = file_content[2], question = question)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at anas-awadalla/gpt-2-large-squad and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at anas-awadalla/gpt-2-large-squad and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.00038174836663529277,
 'start': 7683,
 'end': 7703,
 'answer': ' of issuing stay-at-'}

In [ ]:
#https://huggingface.co/lmqg/t5-small-squad-qag
from transformers import pipeline

pipe = pipeline("text2text-generation", "lmqg/t5-small-squad-qag")
output = pipe(f"{question}, context = {file_content[2]}")[0]['generated_text']
print(output)

Token indices sequence length is longer than the specified maximum sequence length for this model (4645 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Approximately one-third of California's population is Southern California?, | | Southern California


In [ ]:
#https://huggingface.co/deepset/bert-base-cased-squad2
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/bert-base-cased-squad2"
question = "What is southern California often abbreviated as?"
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': question,
    'context': file_content[2]
}
res = nlp(QA_input)
print(res['answer'])


Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


SoCal


In [ ]:
#https://huggingface.co/google-bert/bert-base-uncased
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
question = "What is southern California often abbreviated as?"
# a) Get predictions
nlp = pipeline('question-answering', model=model_name)
QA_input = {
    'question': question,
    'context': file_content[2]
}
res = nlp(QA_input)
print(res)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.00021273687889333814, 'start': 21142, 'end': 21164, 'answer': '000°N 117.000°W\ufeff / 34.'}
